# 03 - Hidden Markov Model Training

Train and evaluate HMM for regime detection

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.models.hmm_model import train_hmm_model, load_hmm_model
from src.models.hmm_utils import analyze_transition_matrix, extract_regime_profiles
from src.viz.plot_regimes import plot_regimes

sns.set_style('whitegrid')
%matplotlib inline

## Load Scaled Features

In [ ]:
df_scaled = pd.read_csv('../data/processed/features_scaled.csv')
df_scaled['date'] = pd.to_datetime(df_scaled['date'])
print(f'Loaded {len(df_scaled)} samples with {df_scaled.shape[1]} features')
df_scaled.head()

## Train HMM

In [ ]:
# Train model with 4 states
hmm_config = {
    'n_iter': 100,
    'random_state': 42,
    'covariance_type': 'full'
}

model, states = train_hmm_model(df_scaled, n_states=4, config=hmm_config)

## Analyze Transition Matrix

In [ ]:
persistence = analyze_transition_matrix(model.transmat_)
print('\nState Persistence:')
for i, p in enumerate(persistence):
    print(f'  State {i}: {p:.3f}')

## Extract Regime Profiles

In [ ]:
df_features = pd.read_csv('../data/processed/features.csv')
df_features['date'] = pd.to_datetime(df_features['date'])

profiles = extract_regime_profiles(df_features, states, model)
profiles

## Visualize Regimes

In [ ]:
df_clean = pd.read_csv('../data/processed/btc_clean.csv')
df_clean['date'] = pd.to_datetime(df_clean['date'])

plot_regimes(df_clean, states, '../results/plots')

## State Statistics

In [ ]:
df_analysis = df_clean.copy()
df_analysis['state'] = states

for state in range(4):
    state_data = df_analysis[df_analysis['state'] == state]
    print(f'\nState {state}:')
    print(f'  Days: {len(state_data)}')
    print(f'  Avg Return: {state_data["return"].mean():.6f}')
    print(f'  Avg Volatility: {state_data["volatility_5d"].mean():.6f}')
    print(f'  Price Range: ${state_data["close"].min():.0f} - ${state_data["close"].max():.0f}')